In [2]:
# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install git+https://github.com/huggingface/datasets.git
!pip install aiohttp

     |████████████████████████████████| 2.5 MB 5.4 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-y93qi3hb
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-y93qi3hb
     |████████████████████████████████| 204 kB 5.3 MB/s 
     |████████████████████████████████| 52 kB 893 kB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 321 kB 39.3 MB/s 
     |████████████████████████████████| 332 kB 49.8 MB/s 
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 981 kB 39.9 MB/s 
     |████████████████████████████████| 3.0 MB 37.2 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 5.6 MB 41.6 MB/s 
     |████████████████████████████████| 100 kB 8.8 MB/s 
     |████████████████████████████████| 8.4 MB 15.3 MB/s 
     |█████████

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-cyvk8ble
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-cyvk8ble
     |████████████████████████████████| 76 kB 74 kB/s 
     |████████████████████████████████| 243 kB 11.3 MB/s 
     |████████████████████████████████| 119 kB 49.3 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
  Created wheel for datasets: filename=datasets-1.11.1.dev0-py3-none-any.whl size=269215 sha256=b98aafa4a1e00fd90964a7db24a718228f93f3de962f5d75e8696bbaecce27e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-hr_u5dj8/wheels/cc/33/c1/2cacc415b23189a83908e45db67381ba26175ef1e8aa9062aa
Successfully built datasets
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface

In [1]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(vector_dim=128, faiss_index_factory_str="Flat")

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
from datasets import load_dataset

# Let's grab wiki snippets dataset from HF
wiki_snippets = load_dataset('wiki_snippets', name='wiki40b_en_100_0', split="train", streaming=True)

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/639 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522M [00:00<?, ?B/s]

Dataset wiki40b downloaded and prepared to /root/.cache/huggingface/datasets/wiki40b/en/1.1.0/3d99a1f96eba780a027018e9a18c1b9a6f5809625e78804a915495f6193dfa5e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def convert_to_doc_dict(article):
  meta = {"name": article['article_title'],"title": article['section_title']}
  return {"text": article['passage_text'], "meta": meta, "embedding":None}


In [4]:
# Now, let's write wiki snippets to our DocumentStore
from tqdm.notebook import tqdm
docs = []
batch_size = 10000
total_doc_count = 100000
with tqdm(total=total_doc_count) as progress_bar:
  for idx, article in enumerate(iter(wiki_snippets)):
    doc = convert_to_doc_dict(article)
    docs.append(doc)
    progress_bar.update(1)
    if idx % batch_size == 0:     
      document_store.write_documents(docs)      
      docs.clear()
    if idx >= total_doc_count:
      break  
    

  0%|          | 0/100000 [00:00<?, ?it/s]

09/13/2021 10:33:58 - INFO - datasets_modules.datasets.wiki_snippets.d152a0e6a420c02b9b26e7f75f45fb54c818cae1d83e8f164f0b1a13ac7998ae.wiki_snippets -   generating examples from = wiki40b en


In [5]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store,
                               embedding_model="yjernite/retribert-base-uncased",
                               model_format="retribert")

document_store.update_embeddings(retriever)

09/13/2021 10:39:59 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model yjernite/retribert-base-uncased
09/13/2021 10:40:00 - INFO - filelock -   Lock 139872936017168 acquired on /root/.cache/huggingface/transformers/f0713cdeb3b97de896c60de3b98c0b5a2fb3747025719a217d8a3387cc71e4e6.af4abfc89963879f7e451f033bc7174a9283ebe41457562d81e71973921777c0.lock


Downloading:   0%|          | 0.00/487 [00:00<?, ?B/s]

09/13/2021 10:40:00 - INFO - filelock -   Lock 139872936017168 released on /root/.cache/huggingface/transformers/f0713cdeb3b97de896c60de3b98c0b5a2fb3747025719a217d8a3387cc71e4e6.af4abfc89963879f7e451f033bc7174a9283ebe41457562d81e71973921777c0.lock
09/13/2021 10:40:00 - INFO - filelock -   Lock 139885370833360 acquired on /root/.cache/huggingface/transformers/39b63af79ebe8963168c4c926c52b942a8f67a8db6d96f3b17f3661666d34b89.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/13/2021 10:40:00 - INFO - filelock -   Lock 139885370833360 released on /root/.cache/huggingface/transformers/39b63af79ebe8963168c4c926c52b942a8f67a8db6d96f3b17f3661666d34b89.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
09/13/2021 10:40:00 - INFO - filelock -   Lock 139885384984784 acquired on /root/.cache/huggingface/transformers/3c21f2e21ff9f8d4c8250f807af5aece2f1c2b43dee6cce65c4fdac6dbc11bf2.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


...

Documents Processed:  80%|████████  | 80000/99994 [57:49<14:27, 23.05 docs/s]

Creating Embeddings:   0%|          | 0/313 [00:00<?, ? Batches/s]

Documents Processed:  90%|█████████ | 90000/99994 [1:05:04<07:13, 23.04 docs/s]

Creating Embeddings:   0%|          | 0/313 [00:00<?, ? Batches/s]

Documents Processed: 100000 docs [1:12:21, 23.04 docs/s]


In [9]:
from haystack.utils import print_answers, print_documents
from haystack.pipeline import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(
    query="Why are Bermudan sloop ships widely prized?",
    #top_k_retriever=10
)
print_documents(res, max_text_len=256)


Query: Why are Bermudan sloop ships widely prized?

{   'name': 'Bermuda sloop',
    'text': 'most prevalent size for such sloops was in the range of 22 to 28 '
            'feet (6.7 to 8.5\xa0m) on deck with a long bowsprit.\n'
            'Jamaican sloops had beams that were narrower than ocean-going '
            'Bermuda sloops, and could attain a speed of around 12 knots. They '
            'carried gaff ...'}

{   'name': 'Bermuda sloop',
    'text': 'vessels it is very much less extreme than on traditional '
            'Bermudian designs, with lower, vertical masts, shorter booms, '
            'omitted bowsprits, and much less area of canvas. Merchant and '
            'privateering use The Bermuda sloop became the predominant type of '
            'sailing v...'}

{   'name': 'Bermuda sloop',
    'text': 'as grain, cocoa, brandy, wine and more from the Atlantic seaboard '
            'colonies to the West Indies.\n'
            'The threat of piracy and privateering was a la

In [10]:
from haystack.generator.transformers import Seq2SeqGenerator
generator = Seq2SeqGenerator(model_name_or_path="yjernite/bart_eli5")

09/13/2021 12:11:10 - INFO - farm.utils -   Using device: CUDA 
09/13/2021 12:11:10 - INFO - farm.utils -   Number of GPUs: 1
09/13/2021 12:11:10 - INFO - farm.utils -   Distributed Training: False
09/13/2021 12:11:10 - INFO - farm.utils -   Automatic Mixed Precision: None


In [11]:
from haystack.pipeline import GenerativeQAPipeline
pipe = GenerativeQAPipeline(generator, retriever)

In [13]:
pipe.run(query="Why are Bermudan sloop ships widely prized?"), #top_k_retriever=4)

({'answers': [' Sloops are a type of sailing ship that can be used for a wide variety of purposes. They can carry a lot of cargo, and they can be very fast. They are also very durable, so they can last a very long time.'],
  'documents': [{'text': 'most prevalent size for such sloops was in the range of 22 to 28 feet (6.7 to 8.5\xa0m) on deck with a long bowsprit.\nJamaican sloops had beams that were narrower than ocean-going Bermuda sloops, and could attain a speed of around 12 knots. They carried gaff rig, whereas in modern usage, a Bermuda sloop excludes any gaff rig. Jamaican sloops were built usually out of cedar trees, for much the same reasons that Bermudian shipwrights favoured Bermuda cedar: these were very resistant to rot, grew very fast and tall, and had a taste displeasing to marine borers. Cedar was favoured', 'score': 0.5589304635076784, 'question': None, 'meta': {'name': 'Bermuda sloop', 'title': 'History', 'vector_id': '36373'}, 'embedding': None, 'id': '2b1da2b5cc2aad

In [15]:
pipe.run(query="Why are woodworking benches usually heavy and rigid?")#, top_k_retriever=4)

{'answers': [" It's not just woodworking benches, it's all kinds of wood. Wood is a very good insulator, and it's very strong. It's also very malleable, which makes it very easy to work with."],
 'documents': [{'text': "- that's really what it's designed for. The old wooden twin-screw design isn't suited for this task because there is no facility for holding the offside jaw open. Construction materials Most workbenches are made from solid wood; the most expensive and desirable are made of solid hardwood. Benches may also be made from plywood and Masonite or hardboard, and bases of treated pine and even steel. There are trade offs with the choice of construction material. Solid wood has many advantages including strength, workability, appearance. A plywood or hardboard bench top has the advantage of being stable, relatively inexpensive, and", 'score': 0.556967221115735, 'question': None, 'meta': {'name': 'Workbench (woodworking)', 'title': 'Face vises as end vises & Construction materia

The answer is not as elaborate as in the previous example. However, we can force the model to generate longer answers. We need to pass the optional parameter min_length (number of words) in the Seq2SeqGenerator constructor. Let's try to do that.


In [16]:
generator = Seq2SeqGenerator(model_name_or_path="yjernite/bart_eli5", min_length=50)
pipe = GenerativeQAPipeline(generator, retriever)

09/13/2021 12:12:14 - INFO - farm.utils -   Using device: CUDA 
09/13/2021 12:12:14 - INFO - farm.utils -   Number of GPUs: 1
09/13/2021 12:12:14 - INFO - farm.utils -   Distributed Training: False
09/13/2021 12:12:14 - INFO - farm.utils -   Automatic Mixed Precision: None


In [17]:
pipe.run(query="Why are woodworking benches usually heavy and rigid?")#, top_k_retriever=4)

{'answers': [" It's not just woodworking benches, it's all kinds of wood. Wood is a very good insulator, and it's very strong. It's also very easy to work with, so it's easy to make it strong and rigid."],
 'documents': [{'text': "- that's really what it's designed for. The old wooden twin-screw design isn't suited for this task because there is no facility for holding the offside jaw open. Construction materials Most workbenches are made from solid wood; the most expensive and desirable are made of solid hardwood. Benches may also be made from plywood and Masonite or hardboard, and bases of treated pine and even steel. There are trade offs with the choice of construction material. Solid wood has many advantages including strength, workability, appearance. A plywood or hardboard bench top has the advantage of being stable, relatively inexpensive, and", 'score': 0.556967221115735, 'question': None, 'meta': {'name': 'Workbench (woodworking)', 'title': 'Face vises as end vises & Construct